In [1]:
#default_exp arcCurves

In [2]:
#export arcCurves
from math import floor, pi
import numpy as np
from osm2xods.utils import giveHeading, distance,schnittpunkt,getXYPositionFromLineLength,getPositiveHeading,getDeltaHdg

length of curve * curvature = delta Heading

<img src="curves.png" width=200/>

In [3]:
#export arcCurves
def getArcEndposition(curvature,length,xstart,ystart,hdg_start):
    deltaHdg = curvature*length
    hdg_end = deltaHdg + hdg_start
    if abs(curvature) > 0.0:
        radius = length/deltaHdg   
        #get Midpoint of Arc
        #turn 90 degree in curvature, walk to the radius to M, then turn back 
        #and substract deltaHeading and walk radius back
        if curvature < 0.0:
            x_M = np.cos(hdg_start+np.pi/2.0)*radius+xstart
            y_M = np.sin(hdg_start+np.pi/2.0)*radius + ystart
            x_end = np.cos(hdg_start-np.pi/2.0+deltaHdg)*radius+x_M
            y_end = np.sin(hdg_start-np.pi/2.0+deltaHdg)*radius+y_M
        else:
            x_M = np.cos(hdg_start+np.pi/2.0)*radius+xstart
            y_M = np.sin(hdg_start+np.pi/2.0)*radius + ystart
            x_end = np.cos(hdg_start-np.pi/2.0+deltaHdg)*radius+x_M
            y_end = np.sin(hdg_start-np.pi/2.0+deltaHdg)*radius+y_M
    else:
        x_end = np.cos(hdg_start)*length +xstart
        y_end = np.sin(hdg_start)*length + ystart
    return x_end, y_end, getPositiveHeading(hdg_end)

In [4]:
print(getArcEndposition(-1,1,0,0,0))
print(getArcEndposition(1,1,0,0,0))
print(getArcEndposition(-0.0000000000001,1,0,0,0))

(0.8414709848078964, -0.45969769413186023, 5.283185307179586)
(0.8414709848078966, 0.45969769413186035, 1.0)
(0.9979760753634936, 0.0, 6.283185307179486)


In [5]:


schnittpunkt(1.0, 1.0, -0.1, -1.0,4.0, -np.pi/2.0)

(-0.9999999999999998,
 1.200669344170901,
 -2.0100418368009105,
 2.799330655829099)

In [6]:
hdg1 = np.pi/2.0
hdg2 = 1
(np.sin(hdg1) * np.cos(hdg2) - np.sin(hdg2) *np.cos(hdg1))

0.5403023058681398

In [22]:
#export arcCurves
def getArcCurvatureAndLength(xstart,ystart,x_end,y_end, x_curveMid, y_curveMid, maxerror = 0.8, minradius = 0.5, iterations = 10):
    #print("Calculating Curve from {0},{1}  over {2},{3} to {4},{5}".format(xstart,ystart,x_curveMid, y_curveMid,x_end,y_end))
    hdg_start = giveHeading(xstart,ystart,x_curveMid, y_curveMid)
    hdg_mid2end = giveHeading(x_curveMid, y_curveMid, x_end,y_end)
    assert hdg_start != hdg_mid2end, "The directions have to be different!"
    deltaHdg = getDeltaHdg(hdg_start,hdg_mid2end)
    #print("deltaHdg = {}".format(deltaHdg))
    winkelHalbHdg = deltaHdg/2.0 + hdg_start
    maxDist = distance(x_curveMid,y_curveMid,xstart,ystart) if distance(x_curveMid,y_curveMid,xstart,ystart) <= \
                distance(x_curveMid,y_curveMid,x_end,y_end) else distance(x_curveMid,y_curveMid,x_end,y_end)
    if abs(deltaHdg) < 0.0001:
        return xstart,ystart,x_end,y_end,0.0,distance(xstart,ystart,x_end,y_end)
    hdg_90_a = hdg_start-np.pi/2.0
    hdg_90_b = hdg_mid2end - np.pi/2.0
        
    #dist = < maxDist
        
    #x1 = x_curveMid + dist*np.cos(hdg_start-np.pi)
    #y1 = y_curveMid + dist*np.sin(hdg_start-np.pi)
    #x2 = x_curveMid + dist*np.cos(hdg_mid2end)
    #y2 = y_curveMid + dist*np.sin(hdg_mid2end)
    #r1 = (-y2 + y1)/((np.sin(hdg_90_a)  + np.sin(hdg_90_b) * (x1 + np.cos(hdg_90_a))/(x2 + np.cos(hdg_90_b)) ))
    #x_m = x1 + np.cos(hdg_90_a) * r1
    #y_m = y1 - np.sin(hdg_90_a) * r1
    #error = ((x_m-x_curveMid)**2 + (y_m-y_curveMid)**2)**0.5  - r1
    # get dist from error...

    dist = maxDist
    bestDist = dist    
    x1 = x_curveMid + dist*np.cos(hdg_start-np.pi)
    y1 = y_curveMid + dist*np.sin(hdg_start-np.pi)
    x2 = x_curveMid + dist*np.cos(hdg_mid2end)
    y2 = y_curveMid + dist*np.sin(hdg_mid2end)
    x_m,y_m,r1,r2 = schnittpunkt(x1,y1,hdg_90_a,x2,y2,hdg_90_b)
    error = distance(x_m,y_m,x_curveMid,y_curveMid) - abs(r1)
    if error < maxerror:
        #print("Error: {0}".format(error))
        pass
    else:
        bestDist = minradius
        notWorkingDist = dist
        for i in range(iterations):
            dist = (bestDist + notWorkingDist)/2.0
            x1 = x_curveMid + dist*np.cos(hdg_start-np.pi)
            y1 = y_curveMid + dist*np.sin(hdg_start-np.pi)
            x2 = x_curveMid + dist*np.cos(hdg_mid2end)
            y2 = y_curveMid + dist*np.sin(hdg_mid2end)
            x_m,y_m,r1,r2 = schnittpunkt(x1,y1,hdg_90_a,x2,y2,hdg_90_b)
            error = distance(x_m,y_m,x_curveMid,y_curveMid) - abs(r1)
            if error < maxerror:
                bestDist = dist
            else:
                notWorkingDist = dist
            #print("BestDistance:{0}  Error: {1}".format(bestDist,error))
    x1 = x_curveMid + bestDist*np.cos(hdg_start-np.pi)
    y1 = y_curveMid + bestDist*np.sin(hdg_start-np.pi)
    x2 = x_curveMid + bestDist*np.cos(hdg_mid2end) 
    y2 = y_curveMid + bestDist*np.sin(hdg_mid2end)
    x_s,y_s,r1,r2 = schnittpunkt(x1,y1,hdg_90_a,x2,y2,hdg_90_b)
    length = abs(r1)*abs(deltaHdg)  #2pi r für vollen kreisbogen
    curvature = -deltaHdg / length
    return x1,y1,x2,y2,curvature,length

# Test the getArcCurvatureAndLength

In [26]:
for hdg1 in np.arange(-3.1,3.1,0.1):
    x1 = np.cos(hdg1-np.pi)*5.0
    y1 = np.sin(hdg1-np.pi)*5.0
    x2 = 0.0
    y2 = 0.0
    for hdg2 in np.arange(-3.1,3.1,0.1):
        if hdg1 != hdg2:
            x3 = np.cos(hdg2)*5.0
            y3 = np.sin(hdg2)*5.0
            x1c,y1c,x3c,y3c,curvature,length = getArcCurvatureAndLength(x1,y1,x3,y3, x2, y2, maxerror = 99.9, minradius = 0.005, iterations = 10)
            xe,ye,hdge = getArcEndposition(curvature,length,x1c, y1c, hdg1)
            if abs(getDeltaHdg(hdge,hdg2)) > 0.001: 
                print("Hdg2 falsch! hdg2:{0}  hdge:{1}".format(hdg2,hdge))
            if abs(x3-xe) > 0.001: 
                print("x end falsch! x3:{0} xe:{1}".format(x3,xe))
                print("hdg1:{0}  hdg2:{1} x1:{2} x1c:{3} x3c:{4} x3:{5} xe:{6} curvature:{7} length:{8}".format(hdg1,hdg2,x1,x1c,x3c,x3,xe,curvature,length))
            if abs(y3-ye) > 0.001: 
                print("y end falsch! y3:{0} ye:{1}".format(y3,ye))
                print("hdg1:{0}  hdg2:{1} y1:{2} y1c:{3} y3c:{4} y3:{5} ye:{6} curvature:{7} length:{8}".format(hdg1,hdg2,y1,y1c,y3c,y3,ye,curvature,length))

In [16]:
xstart,ystart,x_end,y_end, x_curveMid, y_curveMid = 0.0, 0.0,  5.0, 1.0, 3.0, 0.0

getArcCurvatureAndLength(xstart,ystart,x_end,y_end, x_curveMid, y_curveMid, maxerror = 0.8, iterations=10)

(0.7639320225002102,
 -2.7383934913210134e-16,
 5.0,
 1.0,
 0.10557280900008414,
 4.391733187666121)

In [ ]:
getArcEndposition(0.3931147506039497,1.179420533797054, 2.399493463073787, -7.354084083137487e-17, giveHeading(0.0, 0.0,3.0, 0.0))

In [ ]:
giveHeading(3.0, 0.0,5.0, 1.0)

In [ ]:
print(getXYPositionFromLineLength(0.,0.,0,1.))
print(getXYPositionFromLineLength(0.,0.,1,1.))
print(getXYPositionFromLineLength(0.,0.,2,1.))
print(getXYPositionFromLineLength(0.,0.,3,1.))
print(getXYPositionFromLineLength(0.,0.,4,1.))
print(getXYPositionFromLineLength(0.,0.,5,1.))
print(getXYPositionFromLineLength(0.,0.,6,1.))
print(getXYPositionFromLineLength(0.,0.,-1.,1.))

In [ ]:
#export arcCurves
def getArcCurvatureAndLength2Point(xstart,ystart,x_end,y_end, hdg_start, hdg_end):
    if abs(hdg_start - hdg_end) < 0.0001:
        if abs(giveHeading(xstart,ystart,x_end,y_end) - hdg_start) < 0.0001:
            return {"ArcStartCoordinatesXY":[xstart, ystart],"ArcCurvatureLength":[0.0,distance(xstart,ystart,x_end,y_end)],"StartlineLength":0.0,"EndlineLength":0.0,"EndlineStartCoordinatesXY":[x_end, y_end]}
        else:
            #parallele Linien! todo mach nen vernünftigen wert draus
            return {"ArcStartCoordinatesXY":[xstart, ystart],"ArcCurvatureLength":[0.0,distance(xstart,ystart,x_end,y_end)],"StartlineLength":0.0,"EndlineLength":0.0,"EndlineStartCoordinatesXY":[x_end, y_end]}
    hdg_end = hdg_end-np.pi
    # get a synthetic midpoint
    x_s,y_s,r1,r2 = schnittpunkt(xstart,ystart,hdg_start,x_end,y_end,hdg_end)
    #print("schnittpunkt: r1:{0}  r2:{1}".format(r1,r2))
    # 3 cases: either this is the curve-point (r1 and r2 positive) or it is the centerpoint of to diverging lines (r1 and r2 negative)
    # or this is the centerpoint of a new imaginary line (one of both negative, one positive)
    if r1 > 0.0 and r2 > 0.0:
        x1,y1,x2,y2,curvature,length = getArcCurvatureAndLength(xstart,ystart,x_end,y_end, x_s, y_s, maxerror = 999999999.8, minradius = 0.5, iterations = 10)
        return {"ArcStartCoordinatesXY":[x1, y1],"ArcCurvatureLength":[curvature,length],"StartlineLength":distance(xstart,ystart,x1,y1),"EndlineLength":distance(x_end,y_end,x2,y2),"EndlineStartCoordinatesXY":[x2,y2]}
    if r1 < 0.0 and r2 < 0.0:
        # ensure r1 and r2 are the same length (maximum)
        maximumLinelength = max(abs(r1),abs(r2))
        startlineLength = maximumLinelength - abs(r1)
        endlineLength = maximumLinelength - abs(r2) 

    if min(r1,r2)< 0.0 and max(r1,r2) > 0.0:
        #the maximum will have to run forward to the intersection and then again forward with the length of the abs(minimum)
        startlineLength = r1 + abs(r2) if r1 == max(r1,r2) else 0.0
        endlineLength = r2 + abs(r1) if r2 == max(r1,r2) else 0.0
    x_startL, y_startL = getXYPositionFromLineLength(xstart, ystart, hdg_start, startlineLength)
    x_endL, y_endL = getXYPositionFromLineLength(x_end, y_end, hdg_end, endlineLength)
    #get opposite curve - error should be high enough to let the curve start from the beginning points
    _,_,_,_,curvature,length = getArcCurvatureAndLength(x_startL,y_startL,x_endL,y_endL, x_s, y_s, maxerror = 999.8, minradius = 0.5, iterations = 10)
    #get the opposide part of the circle
    deltaHdg = 2*np.pi - abs(length*curvature)
    length = abs(deltaHdg/curvature)
    
    return {"ArcStartCoordinatesXY":[x_startL, y_startL],"ArcCurvatureLength":[curvature,length],"StartlineLength":startlineLength,"EndlineLength":endlineLength,"EndlineStartCoordinatesXY":[x_endL, y_endL]}

In [ ]:
#create a two lane 180 degree turn as a test

In [ ]:
#export arcCurves
def endTurn2LaneStreet(x_start,y_start, hdg_start, preTurnLength = 3.0):
    #create a hard 45° turn in 1 meter
    x_1, y_1, hdg_1 = getArcEndposition(-np.pi/(4.0*preTurnLength),preTurnLength,x_start,y_start,hdg_start)
    x_2, y_2, hdg_2 = getArcEndposition(np.pi/(4.0*preTurnLength),preTurnLength,x_start,y_start,hdg_start)
    hdg_2 += np.pi
    hdg_2 = hdg_2 % (2.*np.pi)
    #get the appropriate curvature
    returnDict = getArcCurvatureAndLength2Point(x_1,y_1,x_2,y_2, hdg_1, hdg_2-np.pi)
    returnDict["StartPointXYHdg"] = [x_start,y_start,hdg_start]
    returnDict["BeginningArcCurvatureLength"] = [-np.pi/(4.0*preTurnLength),preTurnLength]
    returnDict["BeginningArcEndXYHdg"] = [x_1, y_1,hdg_1]
    returnDict["EndLineHdg"] = hdg_2
    returnDict["EndArcBeginningXYHdgCurvatureLength"] = [x_2, y_2,hdg_2,-np.pi/(4.0*preTurnLength),preTurnLength]
    return returnDict

In [ ]:
endTurn2LaneStreet(350.0, 0.0, 0.0)

In [ ]:
def printEndCap(dic):
    length = dic['ArcCurvatureLength'][1] + 2.* dic['BeginningArcCurvatureLength'][1]
    '''
    <road name="Road 2" length="{0}" id="2" junction="-1">
        <link>
            <predecessor elementType="road" elementId="1" contactPoint="start" />
	    <successor elementType="road" elementId="1" contactPoint="start" />
        </link>
        <type s="0.0000000000000000e+0" type="town">
            <speed max="125" unit="mph"/>
        </type>
        <planView>'''.format(length) + '''
            <geometry s="0.0000000000000000e+0" x="{0}" y="{1}" hdg="{2}" length="{3}">
                <arc curvature="{4}"/>
            </geometry>
	    <geometry s="{5}" x="{6}" y="{7}" hdg="{9}"  length="{10}">
                <arc curvature="{11}"/>
            </geometry>
	    <geometry s="{12}" x="{13}" y="{14}" hdg="{15}" length="{16}">
                <arc curvature="{17}"/>
            </geometry>'''.format(350.0, 0.0, 0.0, dic['BeginningArcCurvatureLength'][1])+'''
            
        </planView>
        <elevationProfile>
            <elevation s="0.0000000000000000e+0" a="0.0000000000000000e+0" b="0.0000000000000000e+0" c="0.0000000000000000e+0" d="0.0000000000000000e+0"/>
            
        </elevationProfile>
        <lanes>
            <laneOffset s="0.0000000000000000e+0" a="0.0000000000000000e+0" b="0.0000000000000000e+0" c="0.0000000000000000e+0" d="0.0000000000000000e+0"/>
            <laneSection s="0.0000000000000000e+0">
                <left>
                </left>
                <center>
                    <lane id="0" type="none" level="false">
                        <roadMark sOffset="0.0000000000000000e+0" type="broken" material="standard" color="yellow" width="1.2500000000000000e-1" laneChange="none"/>
                    </lane>
                </center>
                <right>
                    <lane id="-1" type="driving" level="false">
                        <width sOffset="0.0000000000000000e+0" a="4.0000000000000000e+0" b="0.0000000000000000e+0" c="0.0000000000000000e+0" d="0.0000000000000000e+0"/>
                        <roadMark sOffset="0.0000000000000000e+0" type="none" material="standard" color="white" laneChange="none"/>
			<link>
                    		<predecessor id="-1"/>
				<successor id="-1"/>
                        </link>
                        <userData>
                            <vectorLane travelDir="forward"/>
                        </userData>
                    </lane>
                   
                </right>
            </laneSection>
        </lanes>
    </road>'''

In [ ]:
12.42640687119294 + 5